In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from snmfem.conf import DATASETS_PATH

from snmfem.models import EDXS

In [3]:
# Load an EDXS model
model_parameters = {"params_dict" : {"c0" : 4.8935e-05, 
                                          "c1" : 1464.19810,
                                          "c2" : 0.04216872,
                                          "b0" : 0.15910789,
                                          "b1" : -0.00773158,
                                          "b2" : 8.7417e-04},
                         "db_name" : "simple_xrays_threshold.json",
                         "e_offset" : 0.208,
                         "e_scale" : 0.01,
                         "e_size": 1980,
                         "width_slope" : 0.01,
                         "width_intercept" : 0.065,
                         "seed" : 1}
model = EDXS(**model_parameters)
e = model.x

You need to run the script `python script/generate_synthetic_dataset.py` in order to get the data for the next cell.

In [4]:
# load an EDXS sample
sample = DATASETS_PATH / Path("aspim037_N100_2ptcls_brstlg") / Path("sample_0.npz")
# load data
data = np.load(sample)
X = data["X"]
Xdot = data["Xdot"]
nx, ny, ns = X.shape
Xflat = X.transpose([2,0,1]).reshape(ns, nx*ny)
Xdotflat = Xdot.transpose([2,0,1]).reshape(ns, nx*ny)
densities = data["densities"]
phases = data["phases"]
true_spectra = np.expand_dims(densities, axis=1) * phases
true_maps = data["weights"]
k = true_maps.shape[2]
true_maps_flat = true_maps.transpose([2,0,1]).reshape(k,nx*ny)
assert(true_maps.shape[:2] == (nx, ny))
G = data["G"]


In [5]:
A = true_maps.reshape(-1, true_maps.shape[2]).T
GP = np.linalg.lstsq(A.T, Xflat.T)[0]
P = np.linalg.lstsq(G, GP.T)[0]


<ipython-input-5-1ddbb7fba7c4>:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  GP = np.linalg.lstsq(A.T, Xflat.T)[0]
<ipython-input-5-1ddbb7fba7c4>:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(G, GP.T)[0]


In [6]:
# parameters
c0 = 4.8935e-05
c1 = 1464.19810
c2 = 0.04216872
b0 = 0.15910789
b1 = -0.00773158
b2 = 8.7417e-04

# Numpy implementation


In [7]:
def Bmatrix(b0, b1, b2, c0, c1, c2, e):
    beta = b0/e + b1 + b2*e
    Gamma = np.exp(-c2 / e**3) * (1 - np.exp(-c1 / e**3))
    alpha = e**3 * (1 - np.exp(- c0 / e**3)) / c0
    B = np.expand_dims(beta * Gamma * alpha, axis=1 )
    return B

def loss(b0, b1, b2, c0, c1, c2, e, G, P, A, X):
    B = Bmatrix(b0, b1, b2, c0, c1, c2, e)
    GP = G @ P
    GPA = (GP + B) @ A
    return - np.mean(X*np.log(GPA)) + np.mean(GPA)

In [8]:
B = Bmatrix(b0, b1, b2, c0, c1, c2, e)
l = loss(b0, b1, b2, c0, c1, c2, e, G, P, A, Xflat)


# Torch implementation

In [9]:
import torch
from torch.autograd import Variable


In [10]:
t_c0 = Variable(torch.tensor(c0), requires_grad=True)
t_c1 = Variable(torch.tensor(c1), requires_grad=True)
t_c2 = Variable(torch.tensor(c2), requires_grad=True)
t_b0 = Variable(torch.tensor(b0), requires_grad=True)
t_b1 = Variable(torch.tensor(b1), requires_grad=True)
t_b2 = Variable(torch.tensor(b2), requires_grad=True)

t_e = Variable(torch.tensor(e), requires_grad=False)

t_G = Variable(torch.tensor(G), requires_grad=False)
t_P = Variable(torch.tensor(P), requires_grad=False)
t_A = Variable(torch.tensor(A), requires_grad=False)
t_X = Variable(torch.tensor(Xflat), requires_grad=False)


In [11]:
def torch_Bmatrix(b0, b1, b2, c0, c1, c2, e):
    beta = b0/e + b1 + b2*e
    Gamma = torch.exp(-c2 / e**3) * (1 - torch.exp(-c1 / e**3))
    alpha = e**3 * (1 - torch.exp(- c0 / e**3)) / c0
    B = torch.unsqueeze(beta * Gamma * alpha, axis=1 )
    return B

def torch_loss(b0, b1, b2, c0, c1, c2, e, G, P, A, X):
    B = torch_Bmatrix(b0, b1, b2, c0, c1, c2, e)
    GP = torch.matmul(G, P)
    GPA = torch.matmul((GP + B), A )
    return - torch.mean(X*torch.log(GPA)) + torch.mean(GPA)

### Check that the forward functions gives the same results in torch and in numpy

In [12]:
B_torch = torch_Bmatrix(t_b0, t_b1, t_b2, t_c0, t_c1, t_c2, t_e)

np.linalg.norm(B - B_torch.detach().numpy())/np.linalg.norm(B)

1.8819978976686354e-08

In [13]:
l_torch = torch_loss(t_b0, t_b1, t_b2, t_c0, t_c1, t_c2, t_e, t_G, t_P, t_A, t_X)

np.linalg.norm(l - l_torch.detach().numpy())/np.linalg.norm(l)

3.0345401822370927e-09

### Compute the gradients

In [14]:
t_c0.grad = torch.zeros(t_c0.size())
t_c1.grad = torch.zeros(t_c1.size())
t_c2.grad = torch.zeros(t_c2.size())
t_b0.grad = torch.zeros(t_b0.size())
t_b1.grad = torch.zeros(t_b1.size())
t_b2.grad = torch.zeros(t_b2.size())

l_torch = torch_loss(t_b0, t_b1, t_b2, t_c0, t_c1, t_c2, t_e, t_G, t_P, t_A, t_X)
l_torch.backward()

# here they are! The number are quite big indeed. That is likely to cause problem for the optimization...
t_c0.grad, t_c1.grad, t_c2.grad, t_b0.grad, t_b1.grad,t_b2.grad


(tensor(-0.0150),
 tensor(1.5216e-06),
 tensor(-0.0300),
 tensor(0.1057),
 tensor(0.4643),
 tensor(3.7720))

In [ ]:
def chapman_brstlg(E, b0=None, b1=None, b2=None):
        """
        Bremsstrahlung modelling function.
        This function takes arguments to allow calculation with temporary values of the parameters.
        See Chapman et al., 1984, J. of microscopy, vol. 136, pp. 171
        """
        return (b0 / E + b1 + b2 * E) 

def detector(E, c1=None, c2=None):
    """
    Detector modelling function.
    This function takes arguments to allow calculation with temporary values of the parameters.
    Absorption in the dead layer * Photons not absorbed in the detector
    """
    return np.exp(-c2 / np.power(E, 3)) * (
        1 - np.exp(-c1 / np.power(E, 3))
    )

def self_abs(E,c0=None):
    """
    self-absorption modelling function.
    This function takes arguments to allow calculation with temporary values of the parameters.
    Phi rho z model with a constant Phi rho z function
    """
    return (
        np.power(E, 3)
        * (1 - np.exp(-c0 / np.power(E, 3)))
        / c0
    )

def calc_db0 (E,c0,c1,c2) :
    return (1.0/E)* detector(E,c1,c2) * self_abs(E,c0) 

def calc_db1(E,c0,c1,c2):
    """
    Partial derivative of B with respect to b1
    """
    return detector(E,c1,c2) * self_abs(E,c0) 

def calc_db2(E,c0,c1,c2):
    """
    Partial derivative of B with respect to b2
    """
    return E * detector(E,c1,c2) * self_abs(E,c0) 

